# Actuarial Quiz Questions

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import integrate
from scipy.stats import norm

---
## Q1: Integral of Exposure Curve

The exposure curve is defined as:
- For $0 \leq x \leq 1$: $\text{Exposure}(x) = x$
- For $1 < x \leq 2$: $\text{Exposure}(x) = 1 - (x - 1) = 2 - x$

### Step 1: Plot the Exposure Curve

In [ ]:
def exposure(x):
    """Exposure curve: triangular function peaking at x=1."""
    x = np.asarray(x)
    result = np.zeros_like(x, dtype=float)
    mask1 = (x >= 0) & (x <= 1)
    mask2 = (x > 1) & (x <= 2)
    result[mask1] = x[mask1]
    result[mask2] = 2 - x[mask2]
    return result

x_vals = np.linspace(0, 2, 200)
exposure_vals = exposure(x_vals)

plt.figure(figsize=(8, 5))
plt.plot(x_vals, exposure_vals, 'b-', linewidth=2)
plt.xlabel('x')
plt.ylabel('Exposure(x)')
plt.title('Exposure Curve')
plt.grid(True, alpha=0.3)
plt.xlim(0, 2)
plt.ylim(0, 1.1)
plt.show()

### Step 2: Integrate Exposure(x) to get Incurred(x)

$\text{Incurred}(x) = \int_0^x \text{Exposure}(t) \, dt$

In [ ]:
def incurred(x):
    """Incurred: cumulative integral of exposure curve from 0 to x."""
    x = np.asarray(x)
    result = np.zeros_like(x, dtype=float)
    
    # For 0 <= x <= 1: integral of t from 0 to x = x^2/2
    mask1 = (x >= 0) & (x <= 1)
    result[mask1] = x[mask1]**2 / 2
    
    # For 1 < x <= 2: integral from 0 to 1 + integral of (2-t) from 1 to x
    # = 1/2 + [2t - t^2/2] from 1 to x
    # = 1/2 + (2x - x^2/2) - (2 - 1/2)
    # = 1/2 + 2x - x^2/2 - 3/2
    # = 2x - x^2/2 - 1
    mask2 = (x > 1) & (x <= 2)
    result[mask2] = 2*x[mask2] - x[mask2]**2/2 - 1
    
    return result

incurred_vals = incurred(x_vals)
print(f"Incurred(0) = {incurred(np.array([0]))[0]:.4f}")
print(f"Incurred(1) = {incurred(np.array([1]))[0]:.4f}")
print(f"Incurred(2) = {incurred(np.array([2]))[0]:.4f}")

### Step 3: Plot Exposure(x) and Incurred(x) Together

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(x_vals, exposure_vals, 'b-', linewidth=2, label='Exposure(x)')
plt.plot(x_vals, incurred_vals, 'r-', linewidth=2, label='Incurred(x)')
plt.xlabel('x')
plt.ylabel('y')
plt.title('Exposure Curve and Incurred (Integral)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.xlim(0, 2)
plt.ylim(0, 1.1)
plt.show()

---
## Q2: Stop Loss at Sigma Excess of Mean Plus Sigma

Normal distribution with:
- Mean ($\mu$) = 60%
- Standard deviation ($\sigma$) = 6%

Layer: 66% xs 6% (attachment at $\mu + \sigma$, width of $\sigma$)

In [ ]:
mu = 0.60
sigma = 0.06
attachment = 0.66  # mu + sigma
limit = 0.06       # sigma
exhaustion = 0.72  # mu + 2*sigma

dist = norm(loc=mu, scale=sigma)

### Step 1: Approximate Loss Cost (Trapezoidal Rule)

Average of $1 - \text{CDF}(x)$ at 66% and 72%, multiplied by layer width (6%).

In [ ]:
surv_at_66 = 1 - dist.cdf(attachment)
surv_at_72 = 1 - dist.cdf(exhaustion)

approx_loss_cost = (surv_at_66 + surv_at_72) / 2 * limit

print(f"1 - CDF(66%) = {surv_at_66*100:.1f}%")
print(f"1 - CDF(72%) = {surv_at_72*100:.1f}%")
print(f"Average survival probability = {(surv_at_66 + surv_at_72)/2*100:.1f}%")
print(f"\nApproximate loss cost (trapezoidal) = {approx_loss_cost*100:.3f}%")

### Step 2: Integrate 1 - CDF(x) between 66% and 72%

The expected loss to the layer can be computed as:
$E[\text{Layer Loss}] = \int_{66\%}^{72\%} (1 - F(x)) \, dx$

In [ ]:
def survival_function(x):
    return 1 - dist.cdf(x)

integral_result, integral_error = integrate.quad(survival_function, attachment, exhaustion)

print(f"Integrated loss cost = {integral_result*100:.3f}%")
print(f"Integration error estimate = {integral_error:.2e}")

### Step 3: Monte Carlo Simulation

Simulate $x$ 10,000 times and calculate the average of $\min(\max(0, x - 66\%), 6\%)$.

In [ ]:
np.random.seed(42)  # For reproducibility
n_simulations = 10000

simulated_x = dist.rvs(size=n_simulations)
layer_losses = np.minimum(np.maximum(0, simulated_x - attachment), limit)
simulated_loss_cost = np.mean(layer_losses)

print(f"Simulated loss cost (n={n_simulations:,}) = {simulated_loss_cost*100:.3f}%")
print(f"Standard error = {np.std(layer_losses)/np.sqrt(n_simulations)*100:.3f}%")

### Step 4: Compare All Answers

In [ ]:
print("=" * 54)
print("COMPARISON OF METHODS")
print("=" * 54)
print(f"{'Method':<32} {'Loss Cost':>12} {'% Diff':>8}")
print("-" * 54)
print(f"{'1. Trapezoidal Approximation':<32} {approx_loss_cost*100:>11.3f}% {'-':>8}")
print(f"{'2. Numerical Integration':<32} {integral_result*100:>11.3f}% {(integral_result/approx_loss_cost - 1)*100:>+7.1f}%")
print(f"{'3. Monte Carlo Simulation':<32} {simulated_loss_cost*100:>11.3f}% {(simulated_loss_cost/approx_loss_cost - 1)*100:>+7.1f}%")
print("=" * 54)
print(f"\nThe integration method is the most accurate.")
print(f"Trapezoidal rule provides a quick approximation.")
print(f"Simulation converges to the true value with more iterations.")

### Step 5: Simulation Convergence

Demonstrate how the Monte Carlo simulation converges to the numerical integration result as the number of simulations increases.

In [ ]:
np.random.seed(123)  # For reproducibility
max_sims = 20000

# Generate all samples upfront
all_samples = dist.rvs(size=max_sims)
all_layer_losses = np.minimum(np.maximum(0, all_samples - attachment), limit)

# Calculate running average at each point
running_avg = np.cumsum(all_layer_losses) / np.arange(1, max_sims + 1)

# Sample points for plotting (to avoid plotting 20k points)
plot_points = np.concatenate([
    np.arange(1, 100),           # Every point from 1-99
    np.arange(100, 1000, 10),    # Every 10th point from 100-999
    np.arange(1000, max_sims + 1, 100)  # Every 100th point from 1000-20000
])
plot_points = plot_points[plot_points <= max_sims]

plt.figure(figsize=(10, 6))
plt.plot(plot_points, running_avg[plot_points - 1] * 100, 'b-', linewidth=1, label='Simulation Running Average')
plt.axhline(y=integral_result * 100, color='r', linestyle='--', linewidth=2, label=f'Numerical Integration ({integral_result*100:.3f}%)')
plt.xlabel('Number of Simulations')
plt.ylabel('Estimated Loss Cost (%)')
plt.title('Monte Carlo Simulation Convergence')
plt.legend()
plt.grid(True, alpha=0.3)
plt.xlim(0, max_sims)
plt.show()

print(f"Final simulation result (n={max_sims:,}): {running_avg[-1]*100:.3f}%")
print(f"Numerical integration result:            {integral_result*100:.3f}%")
print(f"Difference:                              {abs(running_avg[-1] - integral_result)*100:.3f}%")